In [72]:
%matplotlib inline
import os
import csv
import numpy as np
import re
import datetime
import fnmatch
from pathlib import Path
from itertools import groupby
import pandas as pd
import xlrd


import distance
import nltk
#nltk.download('stopwords')

from nltk.stem import SnowballStemmer
from nltk.stem import snowball

# import other jupyter notebooks
import import_ipynb
from openpyxl import load_workbook

In [21]:
# list of all things that should be present when computing effective time
list_keysToBeCounted = ['Comma', 'BackOne', 'BackMany', 'SpaceBar']

In [4]:
def FixScratchPad(ScratchPad_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    ScratchPad_Times = [item[0] for item in ScratchPad_Old]
    
    ScratchPad_Phrases = list()
    
    # loop to combine phrases divided by commas
    ScratchPadInd = -1 
    while ScratchPadInd < len(ScratchPad_Old)-1:
        ScratchPadInd = ScratchPadInd + 1
        commasInPhrase = len(ScratchPad_Old[ScratchPadInd])-2
        if commasInPhrase < 1:
            #print(ScratchPad_Old[ScratchPadInd][1])
            ScratchPad_Phrases.append(ScratchPad_Old[ScratchPadInd][1])
            continue
        scratchPadPhrase = ScratchPad_Old[ScratchPadInd][1]
        for phraseJoinNr in range(1, commasInPhrase+1):
            scratchPadPhrase = scratchPadPhrase + ', ' + ScratchPad_Old[ScratchPadInd][1+phraseJoinNr]
        
        ScratchPad_Phrases.append(scratchPadPhrase)
            
        
    ScratchPad_New = [[ScratchPad_Times[ind], ScratchPad_Phrases[ind]] for ind in 
                    range(0, len(ScratchPad_Times))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    #print(ScratchPad_New)
    return ScratchPad_New

In [5]:
def FixUserKeys(UserKeys_Old):
    # Fix the situation where comma has divided decimals into separate columns
    
    
    Column_beforeDecimal = [item[2] for item in UserKeys_Old]
    Column_afterDecimal = [item[3] if len(item)>3 else '00' for item in UserKeys_Old]
    
    UserKeys_ProgressPercent = [float(Column_beforeDecimal[i]+'.'+ Column_afterDecimal[i]) for i in 
                                range(0, len(Column_beforeDecimal))]
    UserKeys_Times = [item[0] for item in UserKeys_Old]
    UserKeys_Keys = [item[1] for item in UserKeys_Old]
    
    UserKeys_New = [[UserKeys_Times[ind], UserKeys_Keys[ind], UserKeys_ProgressPercent[ind]] for ind in 
                    range(0, len(UserKeys_ProgressPercent))]
    
    #UserKeys_New = np.concatenate((UserKeys_Times, UserKeys_Keys, UserKeys_ProgressPercent), axis = 0)
    
    
    return UserKeys_New
        

In [14]:
def ComputeDwellTime(userKeys):
    # modify userKeys to include a column of time instead of progress pct, which is dependent on the then dwell time
    
    TimeDwellOrig = 800
    
    timeDwell = TimeDwellOrig
    nKey = -1
    for key in userKeys:
        nKey = nKey + 1
        #print(key[1])
        if key[1] == 'IncreaseDwellTime':
            if float(key[2]) == 1:
                timeDwell = timeDwell + 100
        elif key[1] == 'DecreaseDwellTime':
            #print(key[2])
            if float(key[2]) == 1:
                timeDwell = timeDwell - 100
        else:
            userKeys[nKey].append(str(float(key[2])*timeDwell))
    
    return userKeys

In [7]:
# This function will return the datetime in items which is the closest to the date pivot
def nearestTimePoint(dates, date):
    
    for d in dates:
        if d < date:
            nearestTP = d
        else:
            continue
    try: 
        nearestTP
        nearestTPind = dates.index(nearestTP)
    except:
        nearestTP = 0
        nearestTPind = -1
        
    return nearestTP, nearestTPind

In [46]:
def stimPhrasesEdit_expt2019Dec(PhraseLog, subjName):
    
    # Now extract phrases from the phrase file
    phraseStim_Phrases = [item[1] for item in PhraseLog]
    phraseStim_PhrasesReduced = sorted(set(phraseStim_Phrases), key=phraseStim_Phrases.index)
    
    PhraseLogReduced= list()
    ind = -1
    
    #print(phraseStim_PhrasesReduced)
    
    for i in phraseStim_PhrasesReduced:
        ind = ind + 1

        if ind == len(phraseStim_PhrasesReduced)-1:
            if subjName == 'sa\\Test_woChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:31]])
                
            elif subjName == 'rh\\Test_wChinRest\\p1':
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i[0:-9]])
            else:
                PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
        else:
            PhraseLogReduced.append([phraseLog[phraseStim_Phrases.index(i)][0], i])
            
    del PhraseLogReduced[0]
    del PhraseLogReduced[0]
    

    
    if PhraseLogReduced[-1][1] == 'THE EXPERIMENT IS NOW DONE':
        del PhraseLogReduced[-1]
        
    if 'rh\\Test_wChinRest\\p2' in subjName:
        #print(PhraseLogReduced[2])
        del PhraseLogReduced[2]
        #print(PhraseLogReduced[-1])
        del PhraseLogReduced[-1]
        
    #if subjName == 'sa\\Test_woChinRest\\p1' or subjName == 'rh\\Test_wChinRest\\p1':
    #    del PhraseLogReduced[-1]
        
    if 'sa\\Test_woChinRest\\p2' in subjName:
        del PhraseLogReduced[3]
        del PhraseLogReduced[1]
        del PhraseLogReduced[0]
        
    
        
        
    return PhraseLogReduced

In [54]:
def scratchPadPhraseEdit_expt2019Dec(phraseUser, subjName):
    phraseUserEnd = list()
    
    print(subjName)
    
    #print(phraseUser)
    for row_ind in range(0, len(phraseUser)):
        if row_ind!= 0 and phraseUser[row_ind][1] == '':
            if len(phraseUser[row_ind-1][1])>10:
                phraseUserEnd.append(phraseUser[row_ind-1])
                #print(phraseUser[row_ind-1])
    
    if 'sa\\Test_woChinRest\\p1' in subjName:
        phraseUserEnd.append([phraseUser[-1][0], phraseUser[-1][1][0:35]])
    
    if 'rh\\Test_wChinRest\\p1' in subjName:
        phraseUserEnd.append(phraseUser[-1])
        
    # remove first trial of text composition
    del phraseUserEnd[0]
        
    if 'rh\\Test_wChinRest\\p2' in subjName:
        print('deleting: ', phraseUserEnd[-2])
        del phraseUserEnd[-2]
        
    return phraseUserEnd

In [79]:
def ComputeCorrectLetters(PhraseStim, PhraseUserEnd):
    
    if len(PhraseUserEnd)==len(PhraseStim):
        list_nCorrectLetters = list()
        
        for element in range(0,len(PhraseStim)):
            sentence_phraseStim = PhraseStim[element][1]
            sentence_phraseUser = PhraseUserEnd[element][1]
            
            #print(sentence_phraseStim)
            #print(sentence_phraseUser)
            
            # edit sentence if it has a period at the end:
            #if sentence_phraseStim[-1] == '.':
            #    sentence_phraseStim = sentence_phraseStim[:-1]
            #if sentence_phraseUser[-1] != '.':
            #    sentence_phraseUser = sentence_phraseUser[:-1]
                
            # if pound sign in the sentence:
            # check first if it is in the phraseStim.
            if '£50' in sentence_phraseStim:
                # now if some form of currency (pound/kr/kr.) is typed, that will also be added to the phraseStim
                if 'pounds' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pounds')
                    #print('new', sentence_phraseStim)
                elif 'pound' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 pound')
                    #print('new', sentence_phraseStim)
                
                elif '50kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr.')
                    #print('new', sentence_phraseStim)
                elif '50kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50kr')
                    #print('new', sentence_phraseStim)
                
                elif '50 kr.' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr.')
                    #print('new', sentence_phraseStim)
                elif '50 kr' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', '50 kr')
                    #print('new', sentence_phraseStim)
                
                elif 'L50' in sentence_phraseUser:
                    sentence_phraseStim = sentence_phraseStim.replace('£50', 'L50')
                    #print('new', sentence_phraseStim)
                
                else:
                    sentence_phraseStim = sentence_phraseStim.replace('£', '')
                    #print('new', sentence_phraseStim)
                    
            """
            # if people write [?] instead of the complete sentence
            if '[?]' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('[?]', '')
            
            # if people write .../..../..... instead of the complete sentence
            
            if '.....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('.....', '')
            elif '....' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('....', '')
            elif '...' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('...', '')
            elif '..' in sentence_phraseUser:
                sentence_phraseUser = sentence_phraseUser.replace('..', '')
            """   
            
            #print(sentence_phraseStim)
            #print(sentence_phraseUser)
            #print('\n')
            
            nCorrectLetters = max(len(sentence_phraseStim), len(sentence_phraseUser)) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            #nCorrectLetters = len(sentence_phraseUser) - distance.levenshtein(sentence_phraseStim.lower(), sentence_phraseUser.lower())
            
            list_nCorrectLetters.append(nCorrectLetters)
            #print('Number of correct letters typed: ', nCorrectLetters, '   and number of letters to be typed: ', len(sentence_phraseStim))
            #print('\n')
            
    
    else:
        print('stimulation phrases and user phrases do not match')
        print('stimulation phrases:    ', PhraseStim)
        print('user phrases:    ', PhraseUserEnd)
        
    return list_nCorrectLetters

In [80]:
def EffectiveTimeFromUserKeys(UserKeys, PhrasesStim, PhraseUser, ValidityGazeLog, pathOfSession):
    
    UserKeysTime = [val[0] for val in UserKeys]
    ValidityGazeLog_time = [val[0] for val in ValidityGazeLog]
    
    # session name
    session_folder_name = pathOfSession.split('\\')[-1]
    
    timeTypingList = list()
    
    # time between and within fixations on a single letter
    ind_timePrevious = -1
    for time1 in UserKeysTime[1:]:
        ind_timePrevious = ind_timePrevious + 1
        time0 = UserKeysTime[ind_timePrevious]
        
        time, t1, t2 = time0.partition('+')
        timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
        
        time, t1, t2 = time1.partition('+')
        timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
    
        timeBnPoints = timeEnd - timeStart
        
    
    # find starting point of every phrase 
    #print(len(PhrasesStim), len(PhraseUser))
    for ind_phrase in range(0,len(PhrasesStim)):
        #print(ind, PhrasesStim[ind][1], PhraseUser[ind][1])
        timePhraseStarts = PhrasesStim[ind_phrase][0]
        # Find the time when the phrase starts
        timePhraseStartsWUserKeys, ind_start = nearestTimePoint(UserKeysTime, timePhraseStarts)
        
        
        # find when the phrase ends:
        if ind_phrase == len(PhrasesStim)-1: # if it is the last sentence, reverse index searching starts from the last index
            ind_end = len(UserKeys) - 1
            if 'sa\\Test_woChinRest\\p1' in root:
                ind_end_time, ind_end = nearestTimePoint(UserKeysTime, '2018-12-04T13:47:56.5055472+01:00')
        else: # if it's not the last sentence, reverse index searching starts from the next phrase
            ind_end_time, ind_end = nearestTimePoint(UserKeysTime, PhrasesStim[ind_phrase+1][0])
        
        
        flag_start = 0
        typing_list = list()
        letter_time_list = list()
        letter_list = list()
        keys_all_list = list()
        keys_ind_list = list()
        
        
        
        n_key = 0
        
        # collect the typing letters in a list of lists
        # leftshift do not count, because when comparing stim phrase and user phrase, the sentence is converted to lowercase
        for ind_typing in range(ind_start, ind_end):
            
            # create a list of indexes of added keys, to see later if the time between them should be counted or not
            
            if ind_typing > ind_start:
                if UserKeys[ind_typing-1][1] != UserKeys[ind_typing][1]:
                    n_key = n_key + 1
                    keys_all_list.append(UserKeys[ind_typing][1])
                    
                    
                    
                    
                    
            
            if len(UserKeys[ind_typing][1]) == 1 or UserKeys[ind_typing][1] in list_keysToBeCounted or 'Suggestion' in UserKeys[ind_typing][1]:
                
                #if not UserKeys[ind_typing][1].isdigit(): # time for writing digits need to be counted, since they will 
                # be counted in counting the correct letters
                 
                letter_time_list.append(UserKeys[ind_typing][0]) 
                #print(UserKeys[ind_typing][1])
                if UserKeys[ind_typing][1] != UserKeys[ind_typing+1][1]:
                        
                    # make a huge list of all things continuous to be added
                    #if len(UserKeys[ind_typing+1][1]) != 1 and UserKeys[ind_typing+1][1] not in list_keysToBeCounted and 'Suggestion' not in UserKeys[ind_typing+1][1]:
                        
                    #print(UserKeys[ind_typing][1], ',', UserKeys[ind_typing+1][1])
                    #print('letter time list', letter_time_list)
                    #print(UserKeys[ind_typing][1])
                    #print('')
                    typing_list.append(letter_time_list)
                    keys_ind_list.append(n_key)
                    letter_list.append(UserKeys[ind_typing][1])
                    letter_time_list = list()
                   
        
        # print('start and end:', typing_list[0][0], typing_list[-1][-1])
        time1, t1, t2 = typing_list[0][0].partition('+')
        timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
        time2, t1, t2 = typing_list[-1][-1].partition('+')
        timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
        timeDiff = timeEnd - timeStart
        #print(timeDiff.total_seconds())
        
        # add the time for typing every letter in a phrase
        time_typing_letter = 0
        ind_typing_letter = -1
        
        for letter_typed_list in typing_list:
            ind_typing_letter = ind_typing_letter + 1
            
            #if letter_list[ind_typing_letter] == '.' and ind_typing_letter < len(typing_list)-1: 
            #    if letter_list[ind_typing_letter+1] == '.':
            #        print('next one', letter_list[ind_typing_letter], letter_list[ind_typing_letter+1])
            #    if letter_list[ind_typing_letter-1] == '.':
            #        print('previous one', letter_list[ind_typing_letter-1], letter_list[ind_typing_letter])
            
            #print(letter_list[ind_typing_letter])
            
            time1, t1, t2 = letter_typed_list[0].partition('+')
            timeStart = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time2, t1, t2 = letter_typed_list[-1].partition('+')
            timeEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            timeDiff = timeEnd - timeStart
            
            time_typing_letter = time_typing_letter + timeDiff.total_seconds()
        
            
            # activation time is 250ms - add that for every key looked at, which will not be counted afterwards
            #time_typing_letter = time_typing_letter + 0.25
            
            
            
            #print(timeStart, timeEnd, timeDiff.total_seconds())
        
        timeTypingList.append(time_typing_letter)
        #print(time_typing_letter)

        
        #print(pathOfSession)
        
        
        #print('time in between')
        
        # add time between each letter, if it is valid to add --
        # for data more than 5s, not added at all
        # for data less than 5s, validity checked using gaze origin validity obtain from the tobii tracker, 
        # and if it shows valid pupil data for at least 50% of the time in between
        ind_letterTyped_previous = -1
        time_bnLetterCurrent = 0
        
        
        
        for letter_being_typed in typing_list[1:]:
            
            ind_letterTyped_previous = ind_letterTyped_previous + 1
            
            time_previousEnd_Str = typing_list[ind_letterTyped_previous][-1]
            time_currentStart_Str = letter_being_typed[0]
            
            time1, t1, t2 = time_previousEnd_Str.partition('+')
            time_previousEnd = datetime.datetime.strptime(re.sub('[:.T]','-',time1[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time2, t1, t2 = time_currentStart_Str.partition('+')
            time_currentStart = datetime.datetime.strptime(re.sub('[:.T]','-',time2[:-1]), "%Y-%m-%d-%H-%M-%S-%f")
            
            time_bnLetters_notS = time_currentStart - time_previousEnd
            time_bnLetters = time_bnLetters_notS.total_seconds()
            
            
            if keys_ind_list[ind_letterTyped_previous+1] != keys_ind_list[ind_letterTyped_previous] + 1:
                timeTypingList[-1] = timeTypingList[-1] + 0.25 
                
                #print('activation time at', time_currentStart)
                continue
            
            #print(time_previousEnd, time_currentStart, time_bnLetters, 'between', letter_list[ind_letterTyped_previous], 'and', letter_list[ind_letterTyped_previous+1])
            
            if time_bnLetters < 5:
                
                if time_bnLetters < 2:
                    time_bnLetterCurrent = time_bnLetterCurrent + time_bnLetters
                    
                else:
                    """
                    #if '2019-01-16-17-00-12' in pathOfSession or '2019-01-17-15-31-12_2' in pathOfSession:
                    if session_folder_name in dic_noGazeData:
                        if ind == len(PhrasesStim)-1:
                            return timeTypingList
                        else:
                            continue
                    """
                    # find the number of valid points in gaze
                    timeStart_gazeLog_Str, timeStart_gazeLog_ind = nearestTimePoint(ValidityGazeLog_time, time_previousEnd_Str)
                    timeEnd_gazeLog_Str, timeEnd_gazeLog_ind = nearestTimePoint(ValidityGazeLog_time, time_currentStart_Str)
                    validitySubset = [ValidityGazeLog[ind][1] for ind in range(timeStart_gazeLog_ind, timeEnd_gazeLog_ind)]
                    validity_pct = validitySubset.count(True)/len(validitySubset)
                    
                    if validity_pct > 0.8:
                        time_bnLetterCurrent = time_bnLetterCurrent + time_bnLetters
                    
        #print(timeTypingList[-1], time_bnLetterCurrent)    
        timeTypingList[-1] = timeTypingList[-1] + time_bnLetterCurrent
        #print(timeTypingList[-1])
        
    return timeTypingList

In [81]:
def AddToFile(typing_speed, subjectID, resultPathName):
   
    
    
    wb = load_workbook(resultPathName)
    ws = wb.worksheets[0]
    
    typing_speed.insert(0, subjectID)
    ws.append(typing_speed)
    
    
    
    wb.save(resultPathName)

In [86]:
metricComputed = 'typing_speed'
print(metricComputed)
dataFolderName = r'C:\DTU\Data\201812_ExptToCheckMovementEffect\Data' # accessing external hard disk with the data
a = re.compile('(?<=ExptToCheckMovementEffect\\\\Data\\\\)(.*)(?=\\\\2018-1)')


TypingSpeed = list()

list_typingSpeed_trial = list()
list_typingSpeed_subject = list()

typing_speed_wChinRest = list()
typing_speed_woChinRest = list()

for root, dirs, subfolder in os.walk(dataFolderName):
    
    
    
    if not dirs:
        
        if 'tb' in root or 'trial' in root or 'he\\' in root:
            continue
            
        #if 'rh' not in root:
        #    continue
        
        scratchPad = None
        userKeys = None
        phraseLog = None
        
        for file in subfolder:
            
            if fnmatch.fnmatch(file, 'ScratchPadLog*'):
                try:
                    fScratchPad = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerScratchPad = csv.reader(fScratchPad, quotechar=None)
                    scratchPad = list(readerScratchPad)
                except:
                    if fScratchPad is not None:
                        fScratchPad.close()
                    else:
                        print('error in opening the scratchpad log file')
            
            
            if fnmatch.fnmatch(file, 'user*'):
                try:
                    fUserKey = open(root + '\\' + file, encoding='utf-8',  newline='')
                    readerUserKey = csv.reader(fUserKey, quotechar=None)
                    userKeys = list(readerUserKey)
                    userKeys.remove(userKeys[0])
                except:
                    if fUserKey is not None:
                        fUserKey.close()
                    else:
                        print('error in opening the user key log file')
                        
            if fnmatch.fnmatch(file, 'phrase*'):
                try:
                    fPhraseLog = open(root + '\\' + file, encoding='utf-8')
                    readerPhraseLog = csv.reader(fPhraseLog, quotechar=None)
                    phraseLog = list(readerPhraseLog)
                    
                except:
                    if fPhraseLog is not None:
                        fPhraseLog.close()
                    else:
                        print('error in opening the phrase log file')
                        
            if fnmatch.fnmatch(file, 'ReplacementPhraseLog.csv'):
                
                try:
                    fPhraseLog = open(root + '\\' + file)
                    readerPhraseLog = csv.reader(fPhraseLog)
                    phraseLog = list(readerPhraseLog)
                        
                except:
                    if fPhraseLog is not None:
                        print('closing')
                        fPhraseLog.close()
                    else:
                        print('error in opening the replacement phrase log file')
                        
            if fnmatch.fnmatch(file, 'multiKey*'):
                technique = 'multiKey_selection'
            
            if fnmatch.fnmatch(file, 'tobiiGazeLog*'):
                try:
                    fGazeLog = open(root + '\\' + file, encoding='utf-8', newline='')
                    readerGazeLog = csv.reader(fGazeLog, quotechar=None)
                    gazeLog = list(readerGazeLog)
                    
                except:
                    if fGazeLog is not None:
                        fGazeLog.close()
                    else:
                        print('error in opening the scratchpad log file')
            
                    
                     
        if scratchPad is None or userKeys is None or phraseLog is None or gazeLog is None:
            continue
        else:
            print('subject path', root)
            subjAndSessionName = a.findall(root)[0]
            print('subjand session name', subjAndSessionName)
            subjName = subjAndSessionName.split('\\')[0]
            #print('subject id: ', subjName)
            
            # a participant had problem with the computer, so the last phrase is not documented completely
            if 'sa\\Test_woChinRest\\p1' in root:
                scratchPad = scratchPad[0:-1]
            
            # fix scratchpad due to comma related file changes
            scratchPad_new = FixScratchPad(scratchPad)
            
            # fix phraselog due to comma related file changes
            phraseLog_new = FixScratchPad(phraseLog)
            
            
            # fix userKeys due to comma related file changes
            userKeys_new = FixUserKeys(userKeys)
                
            # find dwell time of typing
            userKeys_wDwellTime = ComputeDwellTime(userKeys_new)
                
            phraseStim_reduced = stimPhrasesEdit_expt2019Dec(phraseLog_new, root)
            
            phraseUserEnd_reduced = scratchPadPhraseEdit_expt2019Dec(scratchPad_new, root)
            
            
            # find validity of gaze log -- combine a validity measure with an or of right and left gaze
            time_gazeLog = [item[0] for item in gazeLog]
            validity_leftGaze = [True if item[2]=='Valid' else False for item in gazeLog ]
            validity_rightGaze = [True if item[9] == 'Valid' else False for item in gazeLog ]
            
            validity_gazeLog = [[time_gazeLog[i], validity_leftGaze[i] or validity_rightGaze[i]] for i in range(0, len(time_gazeLog)) ]
            
            
            """
            print('STIM PHRASES')
            for i in phraseStim_reduced:
                print(i)
            print('USER PHRASES')
            for i in phraseUserEnd_reduced:
                print(i)
            """
            
            if 'p1' in root:
                if 'sa\\Test_woChinRest' in root:
                    phraseStim_reduced[-1][1] = phraseStim_reduced[-1][1][0:36]
                else:
                    phraseStim_reduced[1][1] = phraseStim_reduced[1][1][0:117]
                #print('DONE:::::::', phraseStim_reduced)
                    
                phraseUserEnd_reduced1 = phraseUserEnd_reduced
                phraseStim_reduced1 = phraseStim_reduced
                userKeys_wDwellTime1 = userKeys_wDwellTime
                validity_gazeLog1 = gazeLog
                continue
            
            
            
            if 'p2' in root:
                phraseUserEnd_reduced2 = phraseUserEnd_reduced
                phraseStim_reduced2 = phraseStim_reduced
                userKeys_wDwellTime2 = userKeys_wDwellTime
                validity_gazeLog2 = validity_gazeLog
                
                
                
                phraseUserEnd_reduced = phraseUserEnd_reduced1 + phraseUserEnd_reduced2
                phraseStim_reduced = phraseStim_reduced1 + phraseStim_reduced2
                userKeys_wDwellTime = userKeys_wDwellTime1 + userKeys_wDwellTime2
                validity_gazeLog = validity_gazeLog1 + validity_gazeLog2
                
                phraseUserEnd_reduced1 = list()
                phraseStim_reduced1 = list()
                userKeys_wDwellTime1 = list()
                validity_gazeLog1 = list()
                
            
                
            # check if the number of sentences match in the input and the output
            if len(phraseUserEnd_reduced)!=len(phraseStim_reduced):
                print('needs fixing of number of phrases')
                #print('before', phraseStim_reduced)
                #phraseStim_toMatch, phraseUserEnd_toMatch = MatchPhraseNumbers(phraseStim_reduced, phraseUserEnd_reduced)
                #print('after', phraseStim_toMatch)
            else:
                phraseStim_toMatch = phraseStim_reduced
                phraseUserEnd_toMatch = phraseUserEnd_reduced
                
            
            #for sentence in phraseUserEnd_toMatch:
            #    print(sentence[1])
            
            # Find the number of correct letters typed
            # ComputeCorrectLetters(phraseStim_reduced, phraseUserEnd_reduced)
            nCorrectLetters = ComputeCorrectLetters(phraseStim_toMatch, phraseUserEnd_toMatch)
            print('CORRECT LETTERS', nCorrectLetters)
            
            # Effective time of typing:
            # Find time of typing only every letter, numbers, space and changing of keyboards
            
            effective_time = EffectiveTimeFromUserKeys(userKeys_wDwellTime, phraseStim_toMatch, phraseUserEnd_toMatch, validity_gazeLog, root)
            
            
            if len(nCorrectLetters) == len(effective_time):
                for i in range(0, len(nCorrectLetters)):
                    list_typingSpeed_trial.append((nCorrectLetters[i]*60)/(effective_time[i]*5))
            else:
                print('nCorrectLetters and effective_time have unequal lengths')
            
            
            
            print('effective time', effective_time)
            print('\n')
            
            if 'wChinRest' in root:
                typing_speed_wChinRest.append(list_typingSpeed_trial)
            else:
                typing_speed_woChinRest.append(list_typingSpeed_trial)
                
            list_typingSpeed_trial = list()
            
        


typing_speed
subject path C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\bh\Test_wChinRest\2018-12-6-14-7-18
subjand session name bh\Test_wChinRest
subject id:  bh
C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\bh\Test_wChinRest\2018-12-6-14-7-18
CORRECT LETTERS [71, 113, 38, 54, 52, 66, 48, 58, 36]
effective time [147.019993, 197.15950700000008, 57.04250099999999, 106.64713799999998, 78.46974999999998, 173.95906399999996, 103.183287, 117.808735, 102.09836800000001]


subject path C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\bh\Test_woChinRest\2018-12-4-11-28-41
subjand session name bh\Test_woChinRest
subject id:  bh
C:\DTU\Data\201812_ExptToCheckMovementEffect\Data\bh\Test_woChinRest\2018-12-4-11-28-41
CORRECT LETTERS [89, 101, 96, 96, 48, 43, 54, 100, 74]
effective time [169.25365300000004, 196.62408199999993, 168.31506399999998, 182.1330380000001, 58.912909, 77.74975700000002, 62.482373, 201.79516899999993, 105.34690300000003]


subject path C:\DTU\Data\201812_ExptToCheck

In [85]:
# save data to file

participant_id = [1, 2, 3, 4, 5]
n_trials = 9

participant_id_list = [[1]*n_trials, [2]*n_trials, [3]*n_trials, [4]*n_trials, [5]*n_trials]

# flatten lists
typingSpeed_wChinRest = [item for sublist in typing_speed_wChinRest for item in sublist]
typingSpeed_woChinRest = [item for sublist in typing_speed_woChinRest for item in sublist]

# join lists of with and without chin rest
typingSpeed_total = typingSpeed_wChinRest + typingSpeed_woChinRest

# first find the number of columns in the existing file
filePath_PythonData = "C:\\DTU\\Data\\201812_ExptToCheckMovementEffect\\Data\\python_data2.xlsx"
loc = (filePath_PythonData) 
  
sheet = xlrd.open_workbook(loc).sheet_by_index(0)

# Extracting number of rows 
print(sheet.ncols) 

sheet_columns = [sheet.row(0)[i].value for i in range(0, sheet.ncols)]

df_existing = pd.read_excel(filePath_PythonData)
    
if metricComputed not in sheet_columns:
    filePath_PythonData_new = filePath_PythonData
    print('new metric added')

else:
    filePath_PythonData_new = filePath_PythonData[0:-5] + '_new' + filePath_PythonData[-5:]
    print('This metric is already calculated and saved: ', metricComputed)
    
df_existing[metricComputed] = typingSpeed_total
df_existing.to_excel(filePath_PythonData_new ,index=False)

15
new metric added
